In [2]:
# Kode kraftigt inspireret fra https://www.kaggle.com/apollo2506/ship-detection-using-faster-r-cnn-part-1

# imports
%matplotlib inline
import project_path
from lib.utils import plotutils
from src.data import data_loader

def reload():
    import importlib
    importlib.reload(project_path)
    importlib.reload(plotutils)
    importlib.reload(data_loader)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, random, pickle, json, itertools

from IPython.display import SVG
from tensorflow.keras.utils import plot_model, model_to_dot
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Add, Input, Conv2D, Dropout, Activation, BatchNormalization, MaxPooling2D, ZeroPadding2D, AveragePooling2D, Flatten, Dense)
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import *

print("Ok")

Ok


In [3]:


def conv_block(X, k, filters, stage, block, s=2):
    conv_base_name = 'conv_' + str(stage) + block + '_branch'
    bn_base_name = 'bn_' + str(stage) + block + "_branch"

    F1 = filters

    X = Conv2D(filters=F1, kernel_size=(k, k), strides=(s, s),
               padding='same', name=conv_base_name + '2a')(X)
    X = BatchNormalization(name=bn_base_name + '2a')(X)
    X = Activation('relu')(X)

    return X
    pass

In [4]:

def basic_model(input_shape,classes):

    X_input = Input(input_shape)

    X = ZeroPadding2D((5,5))(X_input)

    X = Conv2D(16,(3,3),strides=(2,2),name='conv1',padding="same")(X)
    X = BatchNormalization(name='bn_conv1')(X)

    # stage 2
    X = conv_block(X,3,32,2,block='A',s=1)
    X = MaxPooling2D((2,2))(X)
    X = Dropout(0.25)(X)

#     Stage 3
    X = conv_block(X,5,32,3,block='A',s=2)
    X = MaxPooling2D((2,2))(X)
    X = Dropout(0.25)(X)

#     Stage 4
    X = conv_block(X,3,64,4,block='A',s=1)
    X = MaxPooling2D((2,2))(X)
    X = Dropout(0.25)(X)

#   Output Layer
    X = Flatten()(X)
    X = Dense(64)(X)
    X = Dropout(0.5)(X)

    X = Dense(128)(X)
    X = Activation("relu")(X)

    X = Dense(classes,activation="softmax",name="fc"+str(classes))(X)

    model = Model(inputs=X_input,outputs=X,name='Feature_Extraction_and_FC')

    return model
    pass

In [5]:
# Get data
shipsData = data_loader.load_ship_data()


Loading ship data from path: C:\Users\victo\Aarhus universitet\Test - Dokumenter\Elektronik\Noter Elektronik\Elektronik 6. semester\ITMAL\ITMAL - Exercises\Aflevering4\src\data\../../data/raw/shipsnet.json
Ship data loaded


In [30]:
# Train/Test/Val split
np.random.seed(42)
np.random.shuffle(shipsData.images)

np.random.seed(42)
np.random.shuffle(shipsData.labels)

total_count = len(shipsData.images)
total_count

train = int(0.7*total_count)
val = int(0.2*total_count)
test = int(0.1*total_count)

train_images, train_labels = shipsData.images[:train], shipsData.labels[:train]
val_images, val_labels = shipsData.images[train:(val+train)], shipsData.labels[train:(val+train)]
test_images, test_labels = shipsData.images[-test:], shipsData.labels[-test:]

train_images.shape, val_images.shape, test_images.shape

((2800, 80, 80, 3), (800, 80, 80, 3), (400, 80, 80, 3))

In [35]:
classWeight = {}

for i in range(0,len(train_labels)):
    classWeight[i] = train_labels[i]+1
    pass
print(classWeight)

{0: 1, 1: 1, 2: 2, 3: 1, 4: 1, 5: 1, 6: 2, 7: 1, 8: 1, 9: 1, 10: 1, 11: 2, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 2, 22: 1, 23: 1, 24: 1, 25: 2, 26: 1, 27: 1, 28: 2, 29: 1, 30: 1, 31: 2, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 2, 44: 1, 45: 1, 46: 2, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 2, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 2, 60: 1, 61: 1, 62: 1, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 2, 76: 2, 77: 1, 78: 1, 79: 2, 80: 1, 81: 2, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 1, 91: 1, 92: 1, 93: 1, 94: 1, 95: 2, 96: 2, 97: 1, 98: 2, 99: 1, 100: 1, 101: 2, 102: 1, 103: 1, 104: 2, 105: 1, 106: 1, 107: 1, 108: 1, 109: 1, 110: 2, 111: 1, 112: 2, 113: 1, 114: 1, 115: 1, 116: 1, 117: 1, 118: 1, 119: 1, 120: 1, 121: 1, 122: 1, 123: 1, 124: 1, 125: 1, 126: 1, 127: 2, 128: 1, 129: 1, 130: 2, 131: 1, 132: 1, 133: 1, 134: 1, 135: 2, 136: 2, 137: 1, 138: 

In [7]:
model = basic_model(input_shape=(80,80,3),classes=2)

plot_model(model,to_file='basic_model.png')
SVG(model_to_dot(model).create(prog='dot',format='svg'))

model.summary()

Model: "Feature_Extraction_and_FC"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 80, 80, 3)]       0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 90, 90, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 45, 45, 16)        448       
_________________________________________________________________
bn_conv1 (BatchNormalization (None, 45, 45, 16)        64        
_________________________________________________________________
conv_2A_branch2a (Conv2D)    (None, 45, 45, 32)        4640      
_________________________________________________________________
bn_2A_branch2a (BatchNormali (None, 45, 45, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 45, 4

In [8]:
opt = Adam(lr=1e-3)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

checkpoint = ModelCheckpoint("model_weights.h5",monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')
logs = TensorBoard("logs")

In [ ]:
epochs = 50
batch_size = 16

history = model.fit(train_images,train_labels,
                   steps_per_epoch=len(train_images)//batch_size,
                   epochs=epochs,
                   verbose=1,
                   validation_data=(val_images,val_labels),
                   validation_steps=len(val_images)//batch_size,
                   callbacks=[checkpoint, logs],
                   class_weight=classWeight # Uncomment if AUGMENTATION is set to FALSE
                   )

Epoch 1/50
175/175 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.4886 

In [22]:
plotutils.show_final_history(history)

AttributeError: module 'matplotlib' has no attribute 'subplots'

In [24]:
val_pred = model.predict(val_images)
val_pred = np.argmax(val_pred,axis=1)
val_pred.shape

val_actual = np.argmax(val_labels,axis=1)

cnf_mat = confusion_matrix(val_actual, val_pred)
np.set_printoptions(precision=2)

plt.figure()
plotutils.plot_confusion_matrix(cnf_mat,classes=class_names)
plt.grid(None)
plt.show();

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
test_pred = model.predict(test_images)
test_pred = np.argmax(test_pred,axis=1)
test_pred.shape

test_actual = np.argmax(test_labels,axis=1)

cnf_mat_test = confusion_matrix(test_actual, test_pred)
np.set_printoptions(precision=2)

plt.figure()
plotutils.plot_confusion_matrix(cnf_mat_test,classes=class_names)
plt.grid(None)
plt.show()

In [11]:
rnd_idx = random.sample(range(0,400),10)

class_labels = {i:class_name for (class_name,i) in class_name_labels.items()}
class_labels

# fig, ax = plt.subplots(2,5,figsize=(5,5))

for i,idx in enumerate(rnd_idx):

    plt.imshow(test_images[idx])
    plt.title("Actual: {}\nPredicted: {}".format(class_labels[test_actual[idx]],class_labels[test_pred[idx]]))
    plt.grid(None)
    plt.show()
    pass

NameError: name 'class_name_labels' is not defined

In [21]:
import inspect
dirname = os.path.dirname(os.path.abspath(inspect.stack()[0][1])) #Current folder

datapath = os.path.join(dirname, '../../models/ship-model.h5')
model.save(datapath)